In [ ]:
%pip install wikipedia langchain_core langchain-ollama langgraph

In [ ]:
from typing import Literal
from langchain_core.tools import tool

@tool
def get_weather(city: Literal["nyc", "sf"]):
    """Use this to get weather information."""
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"
    else:
        raise AssertionError("Unknown city")

tools = [get_weather]




In [ ]:
from langchain_ollama import ChatOllama 
from langgraph.prebuilt import create_react_agent
model = ChatOllama(model="llama3.1")  
agent = create_react_agent(model, tools=tools)

In [ ]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [ ]:
inputs = {"messages": [("user", "What is the weather like in NYC?")]}
print_stream(agent.stream(inputs, stream_mode="values"))

In [ ]:
import wikipedia
from langchain_core.tools import tool

@tool
# Define the Wikipedia search tool
def wikipedia_search(query: str) -> str:
    """Search Wikipedia and return a summary of the first result."""
    try:
        return wikipedia.summary(query, sentences=2)
    except wikipedia.exceptions.DisambiguationError as e:
        return f"Multiple results found: {e.options[:5]}"
    except wikipedia.exceptions.PageError:
        return "No Wikipedia page found."
    
tools = [get_weather, wikipedia_search]

In [ ]:
from langchain_ollama import ChatOllama 
from langgraph.prebuilt import create_react_agent
model = ChatOllama(model="llama3.1")  
agent = create_react_agent(model, tools=tools)

In [ ]:
inputs = {"messages": [("user", "Can you tell me about the Empire state building? and what is the weather like in there?")]}
print_stream(agent.stream(inputs, stream_mode="values"))

In [ ]:
from pydantic import BaseModel, Field


class TripPlan(BaseModel):
    """Respond to the user in this format."""
    city: str =  Literal["New York City", "San Francisco"]
    weather: str = Field(description="The weather in the city")
    days_planned: list[str] = Field(description="List of days planned for the trip")
    activities: list[str] = Field(description="List of activities planned for the trip")

In [ ]:
from langchain_ollama import ChatOllama 
from langgraph.prebuilt import create_react_agent
model = ChatOllama(model="llama3.1")  
agent = create_react_agent(model, tools=tools, response_format=TripPlan)
inputs = {"messages": [("user", "Can you help me plan a trip for 3 day to New York City?")]}
response = agent.invoke(inputs)
response["structured_response"]
